In [3]:
import tensorflow
import stellargraph
import numpy
import matplotlib
import os.path

Please notice, to open jupyter in custom directory use:
>>jupyter notebook --notebook-dir=[your_directory]

Expecting file hierarchy:
    root/
        Antarctica_research_2020/
            .git/
            .gitignore
            readme.md
            notebook.ipynb
        data/
            Sesja 1.xlsx
            .
            .
            Sesja 26.xlsx

In [6]:
data_path = "../data/"
file_list = [name for name in os.listdir(data_path)]
print(file_list)

['Sesja 1.xlsx', 'Sesja 10.xlsx', 'Sesja 11.xlsx', 'Sesja 12.xlsx', 'Sesja 13.xlsx', 'Sesja 14.xlsx', 'Sesja 15.xlsx', 'Sesja 16.xlsx', 'Sesja 17.xlsx', 'Sesja 18.xlsx', 'Sesja 19.xlsx', 'Sesja 2.xlsx', 'Sesja 20.xlsx', 'Sesja 21.xlsx', 'Sesja 22.xlsx', 'Sesja 23.xlsx', 'Sesja 24.xlsx', 'Sesja 25.xlsx', 'Sesja 26.xlsx', 'Sesja 3.xlsx', 'Sesja 4.xlsx', 'Sesja 5.xlsx', 'Sesja 6.xlsx', 'Sesja 7.xlsx', 'Sesja 8.xlsx', 'Sesja 9.xlsx']


In [ ]:

for name in file_list:

In [ ]:
for name in file_list